In [1]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from itertools import product
import time
!pip install kmodes
from kmodes.kmodes import KModes

df = pd.read_csv('DATA/Transport_data_clean.csv')
df.info()
#the idea is to categorize the variables that have more tha 30-50 values to get a more general view and get more valid clusters

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411103 entries, 0 to 411102
Data columns (total 62 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   accident_type                          411103 non-null  int64  
 1   gender                                 411103 non-null  int64  
 2   nationality                            411103 non-null  int64  
 3   work_situation                         411103 non-null  int64  
 4   main_occupation                        411103 non-null  int64  
 5   seniority_in_the_job_months            411103 non-null  int64  
 6   seniority_in_the_job_days              411103 non-null  int64  
 7   contract_type                          411103 non-null  int64  
 8   social_security_regime                 411103 non-null  int64  
 9   economic_activity_center               411103 non-null  int64  
 10  number_workers_center                  411103 non-null  

In [2]:
df = pd.read_csv('DATA/Transport_data_clean.csv')

injury_mapping = {
    1: 'mild',
    2: 'serious',
    3: 'very serious',
    4: 'mortal',
    '404': 'severity not known'
}
df['injury_severity'] = df['injury_severity'].map(injury_mapping).fillna('severity not known')

# Categorize age using labels directly
age_categories = ['16-25', '26-35', '36-45', '46-55', '+56', 'Age not known']
#variables['age'] = pd.Categorical(variables['age'], categories=age_categories, ordered=True)
age_bins = [0, 25, 35, 45, 55, float('inf')]  # Define age bins as numerical ranges
age_labels = ['16-25', '26-35', '36-45', '46-55', '+56']  # Define labels for each age range

# Categorize age using numerical ranges
df['age'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, include_lowest=True)
value_counts = df['age'].value_counts()
percentage = (value_counts / len(df)) * 100
print(percentage)

36-45    33.087815
46-55    27.299485
26-35    21.184959
+56      10.734779
16-25     7.692963
Name: age, dtype: float64


In [3]:
def rename_based_on_start(record_id):
    first_digit = str(record_id)[0]
    if first_digit =='1':
        return 'Electrical problem, explosion, fire'
    elif first_digit =='2':
        return 'Overflow, overturning, leak, spill, emanation'
    elif first_digit =='3':
        return 'Breakage, bursting, slipping, falling, collapsing of Material Agent'
    elif first_digit =='4':
        return 'Loss of total or partial control of work equipment or materials'
    elif first_digit =='5':
        return 'Falling persons – Slip or trip and fall'
    elif first_digit =='6':
        return 'Movement of the body without added physical effort'
    elif first_digit =='7':
        return 'Movement of the body as a result of or with physical effort'
    elif first_digit =='8':
        return 'Surprise, fear, violence, aggression, threat, presence'
    else:
        return 'Other deviations'  # You can handle other cases if needed

# Apply the function to rename the records
df['deviation'] = df['deviation'].apply(rename_based_on_start)

# For demonstration, print the updated DataFrame
value_counts = df['deviation'].value_counts()
percentage = (value_counts / len(df)) * 100
print(percentage)

Movement of the body as a result of or with physical effort            26.897882
Loss of total or partial control of work equipment or materials        24.261073
Movement of the body without added physical effort                     19.104701
Falling persons – Slip or trip and fall                                17.255043
Breakage, bursting, slipping, falling, collapsing of Material Agent     5.045694
Other deviations                                                        3.547043
Surprise, fear, violence, aggression, threat, presence                  2.162718
Overflow, overturning, leak, spill, emanation                           1.335189
Electrical problem, explosion, fire                                     0.390656
Name: deviation, dtype: float64


In [4]:
def rename_based_on_start(record_id):
    first_digit = str(record_id)[0]
    if first_digit =='1':
        return 'Head'
    elif first_digit =='2':
        return 'Neck'
    elif first_digit =='3':
        return 'Back, including spine and thoracolumbar vertebrae'
    elif first_digit =='4':
        return 'Trunk and organs'
    elif first_digit =='5':
        return 'Superior limbs'
    elif first_digit =='6':
        return 'Inferior limbs'
    elif first_digit =='7':
        return 'Whole body - multiple parts'
    else:
        return 'Other injuries'  # You can handle other cases if needed

# Apply the function to rename the records
df['injured_body_part'] = df['injured_body_part'].apply(rename_based_on_start)

# For demonstration, print the updated DataFrame
value_counts = df['injured_body_part'].value_counts()
percentage = (value_counts / len(df)) * 100
print(percentage)

Inferior limbs                                       32.888595
Superior limbs                                       25.653425
Back, including spine and thoracolumbar vertebrae    17.073337
Neck                                                  7.257062
Whole body - multiple parts                           6.885136
Trunk and organs                                      5.219860
Head                                                  4.604685
Other injuries                                        0.417900
Name: injured_body_part, dtype: float64


In [5]:
def rename_based_on_start(record_id):
    first_digit = str(record_id)[0]
    if first_digit =='1':
        return 'Wounds and superficial injuries'
    elif first_digit =='2':
        return 'Bone fractures'
    elif first_digit =='3':
        return 'Dislocations, sprains and strains'
    elif first_digit =='4':
        return 'Traumatic amputations, loss of body parts'
    elif first_digit =='5':
        return 'Shock and internal injuries'
    elif first_digit =='6':
        return 'Burns, scalds and frostbite'
    elif first_digit =='7':
        return 'Poisoning and infections'
    elif first_digit =='8':
        return 'Drowning and suffocation'
    elif first_digit =='9':
        return 'Effects of noise, vibration and pressure'
    else:
        return 'Multiple injuries'  # You can handle other cases if needed

# Apply the function to rename the records
df['injury_description'] = df['injury_description'].apply(rename_based_on_start)

# For demonstration, print the updated DataFrame
value_counts = df['injury_description'].value_counts()
percentage = (value_counts / len(df)) * 100
print(percentage)

Dislocations, sprains and strains            50.749569
Wounds and superficial injuries              30.657281
Bone fractures                                9.505161
Shock and internal injuries                   6.419316
Effects of noise, vibration and pressure      1.284836
Burns, scalds and frostbite                   0.562633
Multiple injuries                             0.477739
Traumatic amputations, loss of body parts     0.242032
Poisoning and infections                      0.068596
Drowning and suffocation                      0.032838
Name: injury_description, dtype: float64


In [6]:
def rename_based_on_start(record_id):
    first_digit = str(record_id)[0]
    if first_digit =='1':
        return 'Machine operations'
    elif first_digit =='2':
        return 'Work with hand tools'
    elif first_digit =='3':
        return 'Driving/being on board a means of transport or cargo equipment'
    elif first_digit =='4':
        return 'Object manipulation'
    elif first_digit =='5':
        return 'Manual transportation'
    elif first_digit =='6':
        return 'Movement'
    elif first_digit =='7':
        return 'Being present'
    else:
        return 'other activities'  # You can handle other cases if needed

# Apply the function to rename the records
df['specific_physical_activity'] = df['specific_physical_activity'].apply(rename_based_on_start)

# For demonstration, print the updated DataFrame
value_counts = df['specific_physical_activity'].value_counts()
percentage = (value_counts / len(df)) * 100
print(percentage)

Movement                                                          28.623484
Driving/being on board a means of transport or cargo equipment    27.969390
Object manipulation                                               20.117343
Manual transportation                                             13.987249
Work with hand tools                                               3.783237
Machine operations                                                 3.012141
Being present                                                      1.649951
other activities                                                   0.857206
Name: specific_physical_activity, dtype: float64


In [7]:

time_bins = [0, 2, 5, float('inf')]  # Define age bins as numerical ranges
time_labels = ['Recently started working', 'Worked between 3 to 6 hours', 'Worked more than 6 hours']  # Define labels for each age range

# Categorize using numerical ranges
df['time_workday'] = pd.cut(df['time_workday'], bins=time_bins, labels=time_labels, include_lowest=True)

timeday_bins = [0, 8, 13, 20, float('inf')]  # Define age bins as numerical ranges
timeday_labels = ['Dawn', 'Morning', 'Afternoon','Night']  # Define labels for each age range

# Categorize using numerical ranges
df['time_day'] = pd.cut(df['time_day'], bins=timeday_bins, labels=timeday_labels, include_lowest=True)

seniority_bins = [0, 12, 36, 120 ,float('inf')]  # Define age bins as numerical ranges
seniority_labels = ['New hires', 'Junior employees', 'Experienced', 'Veterans']  # Define labels for each age range

# Categorize using numerical ranges
df['seniority_in_the_job_months'] = pd.cut(df['seniority_in_the_job_months'], bins=seniority_bins, labels=seniority_labels, include_lowest=True)

value_counts = df['seniority_in_the_job_months'].value_counts()
percentage = (value_counts / len(df)) * 100
print(percentage)

New hires           36.075631
Experienced         25.839996
Veterans            20.376889
Junior employees    17.707484
Name: seniority_in_the_job_months, dtype: float64


In [8]:
#df.to_csv('DATA\categorized_data_clean.csv', index=False)
#df = pd.read_csv('DATA/Transport_data_clean.csv')
m
